In [ ]:
import json
import folium
from folium.plugins import MarkerCluster
import requests
from time import sleep, time, gmtime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
import operator

host = "https://5c74a7a1-d505-4b2b-a4fa-efea0c0f20da.deepnoteproject.com"


In [ ]:
# Hotpital Loctaions
coordinates = {
    "MariaVittoria":([45.08192013944934, 7.656257052492525], "CitTurin"),
    "MaurizianoUmberto":([45.05282903776175, 7.665522250053315], "Crocetta"),
    "Koelliker": ([45.04250727636271, 7.654525513659431], "SantaRita"),
    "AmedeoDiSavoia":([45.08949813169014, 7.659568126952876],"CitTurin"),
    "FornacaClinic": ([45.0655322474048, 7.6699852668526995], "Crocetta"),
    "ReginaMargherita":([45.03591129022684, 7.674300759157799], "Lingotto"),
    "Martini":([45.06779437901263, 7.628387811267748], "PozzoStrada"),
    "ClinicaPinnaPintor": ([45.05824164194012, 7.658723111042297], "Crocetta"),
    "Molinette":([45.04243989091534, 7.673989903546069], "Lingotto")
}



In [ ]:
#Gets real time the location of a hospital in the last 2 seconds
end = int(time()*1000)
# start = end-2000
start = 0
response = requests.get(host + '/sensor' + f"?from={start}&to={end}")
print(json.loads(response.content.decode()))

if response.ok:
    responseDict = json.loads(response.content.decode())
    mappa=folium.Map(location=[45.08192013944934, 7.656257052492525], zoom_start=13)
    for sensor in responseDict.keys():
        if len(responseDict[sensor]["timestamps"]) > 0:
            folium.Marker(coordinates[sensor][0], popup=f"<i>{sensor}</i>").add_to(mappa)

else:
    print(response.text)

mappa

{'FornacaClinic': {'timestamps': [1688653805306, 1688653819281], 'stutteringEvents': [1.0, 1.0]}, 'Martini': {'timestamps': [1688653771766, 1688653806700, 1688653814391], 'stutteringEvents': [1.0, 1.0, 1.0]}, 'ClinicaPinnaPintor': {'timestamps': [1688653791758, 1688653817851, 1688653826382], 'stutteringEvents': [1.0, 1.0, 1.0]}, 'AmedeoDiSavoia': {'timestamps': [1688653789815, 1688653823866], 'stutteringEvents': [1.0, 1.0]}, 'MariaVittoria': {'timestamps': [1688503795609, 1688503851516], 'stutteringEvents': [1.0, 1.0]}, 'Koelliker': {'timestamps': [1688653811370], 'stutteringEvents': [1.0]}}


In [ ]:
#Gets statistics in terms of number of hospitals
end = int(time()*1000)
start = 0
response = requests.get(host + '/sensor' + f"?from={start}&to={end}")

if response.ok:
    responseDict = json.loads(response.content.decode())
    is_hospital = pd.DataFrame()

    for sensor in responseDict.keys():
        if len(responseDict[sensor]["timestamps"]) > 0:
            for ts, es in zip(responseDict[sensor]["timestamps"],responseDict[sensor]["stutteringEvents"]):
                is_hospital = is_hospital.append({
                "sensor": sensor,
                "timestamps": ts,
                "stutteringEvents": es,
            }, ignore_index = True)
        #Manipulation to add data and set representation granularity in time
        is_hospital['quartiere'] = is_hospital["sensor"].map(lambda s: coordinates[s][1])
        is_hospital['year'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_year)
        is_hospital['month'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_mon)
        is_hospital['day'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_mday)
        is_hospital['year'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_year)
        is_hospital['weekday'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_wday)
        is_hospital['hour'] = is_hospital["timestamps"].map(lambda t: gmtime(t/1000).tm_hour)


else:
    print(response.text)

In [ ]:
DeepnoteChart(is_hospital, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"sensor","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"count","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [ ]:
#Delete and check sensors content
#requests.delete(host+"/delete")
dictionary = requests.get(host + "/list")
print(json.loads(dictionary.content.decode()))

{'sensor': ['FornacaClinic', 'Martini', 'ClinicaPinnaPintor', 'AmedeoDiSavoia', 'MariaVittoria', 'Koelliker']}


In [ ]:
def zone_min_max_hospitals(start, end, min_max):
    '''gets a timerange and displays the location with the lowest or highest (min_max) number of stutteringEvents'''
    response = requests.get(host + '/sensor' + f"?from={start}&to={end}")
    if response.ok:
        responseDict = json.loads(response.content.decode())
        sensor_hospitals={}
        for sensor in responseDict.keys():
            num_hospitals = sum(responseDict[sensor]['stutteringEvents'])
            sensor_hospitals[sensor]=num_hospitals
        
        if(min_max=='max'):
            x = max(sensor_hospitals.items(), key=operator.itemgetter(1))
            mappa=folium.Map(location=coordinates[x[0]][0], zoom_start=15)
            folium.Marker(coordinates[x[0]][0], popup=f"<i>{x[1]}</i>",icon=folium.Icon(color="red")).add_to(mappa)
        else:
            x= min(sensor_hospitals.items(), key=operator.itemgetter(1))
            mappa=folium.Map(location=coordinates[x[0]][0], zoom_start=15)   
            folium.Marker(coordinates[x[0]][0], popup=f"<i>{x[1]}</i>",icon=folium.Icon(color="green")).add_to(mappa)      
    else:
        print(response.text)
    return mappa


In [ ]:
zone_min_max_hospitals(0, int(time()*1000),'max')

In [ ]:
zone_min_max_hospitals(0, int(time()*1000),'min')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>